# Reinforcement Learning

## The Game 
What is a state and what is an action?

In [5]:
import gym
from IPython.display import clear_output
import time

env=gym.make('Taxi-v3')
env.reset()
score = 0
for _ in range(30):
    env.render()
    action = env.action_space.sample()
    print(action)
    observation, reward, done, info = env.step(action) # take a random action    
    score += reward
    print(score, reward)
    print(observation, reward, done, info)
    time.sleep(0.5)
    clear_output(True)
env.render()
print(score)
env.close()
print(env.env.s) # how to get current state
print(env.action_space)
print(env.observation_space)
#env.reward

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
-147
444
Discrete(6)
Discrete(500)


## QTable? 

### What does each element represent?

### What is the dimension of QTable?

### If we are at state $s$ how do we pick the next best action

Closing Note: QTable doesn't really have to be a table. You can think about it as a parametrized function. The discrete one is much easier to implement.

## Q Learning

### Write down the update rule

### What is $\alpha$?
What happen when alpha is 1 and what happen when alpha is 0?

### What is gamma?

### What value does the update rule converge to?

### When learning do we always want to use the best action?

### What is $\epsilon$?

### When we evaluate the QTable do we want to always use the best action?

In [6]:
print(env.action_space.n)
print(env.observation_space)

6
Discrete(500)


## Build QLearning to play the taxi game

In [8]:
import numpy as np
class QTable:
    def __init__(self, qtable: np.ndarray):
        self.qtable = qtable
      
    def update_table(self, s:int, a:int, reward: float, gamma: float, alpha: float):
        """update q table to a better value"""
        pass
    
    def best_action(self, s: int) -> int:
        """find the best action for the given state"""
    
    def learn(self, env):
        """Iteratively update QTable"""
        pass
    
    @classmethod
    def for_env(self, env):
        n_state = env.action_space.n
        n_action = env.observation_space.n
        return QTable(np.zeros(n_state, n_action))